here we have to verify we're training correctly. we'll train manually and using the model file.

In [51]:
from itertools import product
from functools import partial
import datetime as dt
import pandas as pd
import satori
import numpy as np

In [60]:
import os
import random 
import joblib
import numpy as np
import pandas as pd
from itertools import product
from functools import partial
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import ppscore
from xgboost import XGBRegressor


In [3]:
try:
    df = pd.read_csv("../../data/history/EUR.csv", index_col=0)
except:
    pass
    #df = satori.apis.fx.compileHistory(
    #    name='EUR', 
    #    days=8000, 
    #    cooldown=0, 
    #    saveDuring=True)
df.head(2)

,USD,CHF,GBP,CAD,AUD,NZD,JPY,year,month,day,weekday
2000-01-22,1.0097,1.6102,0.6113,1.4607,1.5215,1.9623,105.53,2000,1,22,5
2000-01-23,1.0097,1.6102,0.6113,1.4607,1.5215,1.9623,105.53,2000,1,23,6


In [4]:
df.columns

Index(['USD', 'CHF', 'GBP', 'CAD', 'AUD', 'NZD', 'JPY', 'year', 'month', 'day',
       'weekday'],
      dtype='object')

In [5]:
targets = ['USD', 'CHF', 'GBP', 'CAD', 'AUD', 'NZD', 'JPY',]
times = ['year', 'month', 'day', 'weekday',]

In [6]:
# drop rows if targets are nan
df = df.drop(index=df[df[targets].isna().any(axis=1)].index)

In [7]:
# drop all but targets and times
df = df[[*targets, *times]]
df.tail(2)

,USD,CHF,GBP,CAD,AUD,NZD,JPY,year,month,day,weekday
2022-01-14,1.1447,1.0429,0.83508,1.433,1.5803,1.6745,130.17,2022,1,14,4
2022-01-15,1.1447,1.0429,0.83508,1.433,1.5803,1.6745,130.17,2022,1,15,5


In [8]:
kwargs = {
    'hyperParameters': [
        satori.HyperParameter(
            name='n_estimators',
            value=300,
            kind=int,
            limit=100,
            minimum=200,
            maximum=5000),
        satori.HyperParameter(
            name='learning_rate',
            value=0.3,
            kind=float,
            limit=.05,
            minimum=.01,
            maximum=.1),
        satori.HyperParameter(
            name='max_depth',
            value=6,
            kind=int,
            limit=1,
            minimum=2,
            maximum=10),
        #satori.HyperParameter(
        #    name='gamma',
        #    value=0,
        #    kind=float,
        #    limit=.05,
        #    minimum=0,
        #    maximum=1),
        #satori.HyperParameter(
        #    name='min_child_weight',
        #    value=1,
        #    kind=float,
        #    limit=.1,
        #    minimum=0,
        #    maximum=5),
        #satori.HyperParameter(
        #    name='max_delta_step',
        #    value=0,
        #    kind=float,
        #    limit=.01,
        #    minimum=0,
        #    maximum=1),
        #satori.HyperParameter(
        #    name='scale_pos_weight',
        #    value=1,
        #    kind=float,
        #    limit=.1,
        #    minimum=0,
        #    maximum=5),
        #satori.HyperParameter(
        #    name='base_score',
        #    value=.5,
        #    kind=float,
        #    limit=.05,
        #    minimum=0,
        #    maximum=1),
    ],
    'metrics':  {
        # raw data features
        'raw': satori.ModelManager.rawDataMetric,
        # daily percentage change, 1 day ago, 2 days ago, 3 days ago... 
        **{f'Daily{i}': partial(satori.ModelManager.dailyPercentChangeMetric, yesterday=i) for i in list(range(1, 31))},
        # rolling period transformation percentage change, max of the last 7 days, etc... 
        **{f'Rolling{i}{tx[0:3]}': partial(satori.ModelManager.rollingPercentChangeMetric, window=i, transformation=tx)
            for tx, i in product('sum() max() min() mean() median() std()'.split(), list(range(2, 21)))},
        # rolling period transformation percentage change, max of the last 50 or 70 days, etc... 
        **{f'Rolling{i}{tx[0:3]}': partial(satori.ModelManager.rollingPercentChangeMetric, window=i, transformation=tx)
            for tx, i in product('sum() max() min() mean() median() std()'.split(), list(range(22, 90, 7)))}},
    'override': True}

In [9]:
models = {
        satori.ModelManager(
            modelPath=f'../../models/EUR{currency}New.jobLib',
            chosenFeatures=[f'Raw{currency}'],
            pinnedFeatures=[f'Raw{currency}'],
            targetKey=currency,
            **kwargs) 
        for currency in targets
    }

In [25]:
list(models)[6].data

,High,Low,Close
0,0.837240,0.830560,0.83577
1,0.837100,0.825830,0.82720
2,0.828020,0.824400,0.82488
3,0.830290,0.823450,0.82775
4,0.828780,0.820280,0.82055
...,...,...,...
4590,0.846070,0.843668,0.84560
4591,0.849574,0.846200,0.84660
4592,0.847314,0.844096,0.84660
4593,0.847200,0.844800,0.84660


In [22]:
model = satori.ModelManager(
            modelPath=f'../../models/EURUSDNew.jobLib',
            chosenFeatures=[f'RawUSD'],
            pinnedFeatures=[f'RawUSD'],
            targetKey='USD',
            **kwargs) 
        


In [24]:
model.data

,High,Low,Close
0,0.837240,0.830560,0.83577
1,0.837100,0.825830,0.82720
2,0.828020,0.824400,0.82488
3,0.830290,0.823450,0.82775
4,0.828780,0.820280,0.82055
...,...,...,...
4590,0.846070,0.843668,0.84560
4591,0.849574,0.846200,0.84660
4592,0.847314,0.844096,0.84660
4593,0.847200,0.844800,0.84660


In [28]:
model.chosenFeatures

['RawUSD']

In [37]:
model.dataPath

'data.parquet'

In [30]:
model.produceFeatureSet()

In [33]:
model.features.get('RawUSD')

In [34]:
model.buildStable()

KeyError: 'USD'

In [36]:
model.data

,High,Low,Close
0,0.837240,0.830560,0.83577
1,0.837100,0.825830,0.82720
2,0.828020,0.824400,0.82488
3,0.830290,0.823450,0.82775
4,0.828780,0.820280,0.82055
...,...,...,...
4590,0.846070,0.843668,0.84560
4591,0.849574,0.846200,0.84660
4592,0.847314,0.844096,0.84660
4593,0.847200,0.844800,0.84660


In [74]:
# target
target = pd.DataFrame(model.data.loc[:, 'High'].shift(-1))
target

,RawHigh
0,0.837100
1,0.828020
2,0.830290
3,0.828780
4,0.822770
...,...
4590,0.849574
4591,0.847314
4592,0.847200
4593,0.828800


In [55]:
# feature
features = [model.features.get('RawHigh')(model.data)]
features

[0       0.837240
 1       0.837100
 2       0.828020
 3       0.830290
 4       0.828780
           ...   
 4590    0.846070
 4591    0.849574
 4592    0.847314
 4593    0.847200
 4594    0.828800
 Name: RawHigh, Length: 4595, dtype: float64]

In [56]:
featureSet = pd.concat(features, axis = 1, keys = ['RawHigh'])
featureSet

,RawHigh
0,0.837240
1,0.837100
2,0.828020
3,0.830290
4,0.828780
...,...
4590,0.846070
4591,0.849574
4592,0.847314
4593,0.847200


In [57]:
# predictable
current = pd.DataFrame(featureSet.iloc[-1, :]).T.dropna(axis=1)
current

,RawHigh
4594,0.8288


In [65]:
df = featureSet.copy()

In [66]:
df = df.iloc[0:-1, :].replace([np.inf, -np.inf], np.nan).reset_index(drop=True)
df

,RawHigh
0,0.837240
1,0.837100
2,0.828020
3,0.830290
4,0.828780
...,...
4589,0.847020
4590,0.846070
4591,0.849574
4592,0.847314


In [306]:
data = df
labels = target.iloc[0:df.shape[0], :]

In [296]:
trainX, testX, trainY, testY = train_test_split(
    df, target.iloc[0:df.shape[0], :], test_size=.5, shuffle=False)

In [297]:
pd.concat([trainX, trainY], axis=1, keys=['trainX', 'trainY'])

,trainX,trainY
,RawHigh,RawHigh
0,0.837240,0.837100
1,0.837100,0.828020
2,0.828020,0.830290
3,0.830290,0.828780
4,0.828780,0.822770
...,...,...
2292,0.772000,0.764100
2293,0.764100,0.764374
2294,0.764374,0.768280


In [298]:
pd.concat([testX, testY], axis=1, keys=['testX', 'testY'])

,testX,testY
,RawHigh,RawHigh
2297,0.771992,0.773772
2298,0.773772,0.772550
2299,0.772550,0.775690
2300,0.775690,0.775800
2301,0.775800,0.775930
...,...,...
4589,0.847020,0.846070
4590,0.846070,0.849574
4591,0.849574,0.847314


In [299]:
xgb = XGBRegressor(
    **{
        param.name: param.value 
        for param in model.hyperParameters 
        if param.name != 'n_estimators'}, 
    n_estimators=3000)

In [307]:
xgb.fit(
    data,
    labels,
    eval_set=[(data, labels)],
    eval_metric='mae', 
    early_stopping_rounds=128,
    verbose=True)

[0]	validation_0-mae:0.21366
[1]	validation_0-mae:0.14969
[2]	validation_0-mae:0.10488
[3]	validation_0-mae:0.07350
[4]	validation_0-mae:0.05153
[5]	validation_0-mae:0.03614
[6]	validation_0-mae:0.02536
[7]	validation_0-mae:0.01781
[8]	validation_0-mae:0.01256
[9]	validation_0-mae:0.00895
[10]	validation_0-mae:0.00657
[11]	validation_0-mae:0.00505
[12]	validation_0-mae:0.00412
[13]	validation_0-mae:0.00359
[14]	validation_0-mae:0.00329
[15]	validation_0-mae:0.00312
[16]	validation_0-mae:0.00303
[17]	validation_0-mae:0.00297
[18]	validation_0-mae:0.00292
[19]	validation_0-mae:0.00290
[20]	validation_0-mae:0.00288
[21]	validation_0-mae:0.00287
[22]	validation_0-mae:0.00286
[23]	validation_0-mae:0.00284
[24]	validation_0-mae:0.00284
[25]	validation_0-mae:0.00283
[26]	validation_0-mae:0.00282
[27]	validation_0-mae:0.00281
[28]	validation_0-mae:0.00280
[29]	validation_0-mae:0.00279
[30]	validation_0-mae:0.00279
[31]	validation_0-mae:0.00278
[32]	validation_0-mae:0.00277
[33]	validation_0-ma

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.3, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=3000, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [304]:
xgb.fit(
    trainX,
    trainY,
    eval_set=[(trainX, trainY), (testX, testY)],
    eval_metric='mae', 
    early_stopping_rounds=128,
    verbose=True)

[0]	validation_0-mae:0.18133	validation_1-mae:0.25900
[1]	validation_0-mae:0.12708	validation_1-mae:0.19352
[2]	validation_0-mae:0.08910	validation_1-mae:0.14632
[3]	validation_0-mae:0.06250	validation_1-mae:0.11204
[4]	validation_0-mae:0.04387	validation_1-mae:0.08818
[5]	validation_0-mae:0.03083	validation_1-mae:0.07024
[6]	validation_0-mae:0.02167	validation_1-mae:0.05787
[7]	validation_0-mae:0.01527	validation_1-mae:0.04833
[8]	validation_0-mae:0.01083	validation_1-mae:0.04156
[9]	validation_0-mae:0.00785	validation_1-mae:0.03667
[10]	validation_0-mae:0.00594	validation_1-mae:0.03348
[11]	validation_0-mae:0.00477	validation_1-mae:0.03100
[12]	validation_0-mae:0.00409	validation_1-mae:0.02908
[13]	validation_0-mae:0.00372	validation_1-mae:0.02795
[14]	validation_0-mae:0.00348	validation_1-mae:0.02698
[15]	validation_0-mae:0.00337	validation_1-mae:0.02620
[16]	validation_0-mae:0.00329	validation_1-mae:0.02598
[17]	validation_0-mae:0.00324	validation_1-mae:0.02549
[18]	validation_0-ma

[149]	validation_0-mae:0.00266	validation_1-mae:0.02426
[150]	validation_0-mae:0.00266	validation_1-mae:0.02426
[151]	validation_0-mae:0.00266	validation_1-mae:0.02426
[152]	validation_0-mae:0.00266	validation_1-mae:0.02426
[153]	validation_0-mae:0.00266	validation_1-mae:0.02426
[154]	validation_0-mae:0.00266	validation_1-mae:0.02426
[155]	validation_0-mae:0.00266	validation_1-mae:0.02426
[156]	validation_0-mae:0.00266	validation_1-mae:0.02426
[157]	validation_0-mae:0.00266	validation_1-mae:0.02426
[158]	validation_0-mae:0.00266	validation_1-mae:0.02426
[159]	validation_0-mae:0.00266	validation_1-mae:0.02426
[160]	validation_0-mae:0.00266	validation_1-mae:0.02426
[161]	validation_0-mae:0.00266	validation_1-mae:0.02426
[162]	validation_0-mae:0.00266	validation_1-mae:0.02426
[163]	validation_0-mae:0.00266	validation_1-mae:0.02426
[164]	validation_0-mae:0.00266	validation_1-mae:0.02426
[165]	validation_0-mae:0.00266	validation_1-mae:0.02426
[166]	validation_0-mae:0.00266	validation_1-mae:

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.3, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=3000, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [310]:
evals_result = xgb.evals_result()


In [311]:
yesterday

,RawHigh
4592,0.847314


In [312]:
today

,RawHigh
4593,0.8472


In [313]:
current

,RawHigh
4594,0.8288


In [315]:
xgb.predict(today)#, iteration_range=(7,56))

array([0.8387696], dtype=float32)

[299]	validation_0-mae:0.00253	validation_1-mae:0.00292
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.3, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=300, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [106]:
#20 %
xgb.predict(current)

array([0.8284466], dtype=float32)

In [112]:
#10 %
xgb.predict(current)

array([0.8275409], dtype=float32)

In [118]:
#1 %
xgb.predict(current)

array([0.82887274], dtype=float32)

In [127]:
#0 %
xgb.predict(current)

array([0.8291307], dtype=float32)

In [135]:
#0 % no eval set
xgb.predict(current)

array([0.8291307], dtype=float32)

In [136]:
current

,RawHigh
4594,0.8288


In [172]:
testX

,RawHigh
4592,0.847314
4593,0.847200


In [170]:
yesterday = pd.DataFrame(testX.iloc[0]).T

In [292]:
yesterday

,RawHigh
4592,0.847314


In [293]:
#today = pd.DataFrame(testX.iloc[-1]).T
today

,RawHigh
4593,0.8472


In [173]:
#0 % no eval set
xgb.predict(yesterday)

array([0.8494901], dtype=float32)

In [179]:
#20 %
xgb.predict(yesterday)

array([0.8475488], dtype=float32)

In [185]:
#10 %
xgb.predict(yesterday)

array([0.8458524], dtype=float32)

In [191]:
#1 %
xgb.predict(yesterday)

array([0.8482324], dtype=float32)

In [197]:
#50 %
xgb.predict(yesterday)

array([0.8471888], dtype=float32)

In [199]:
#50 %
xgb.predict(yesterday)

array([0.8471888], dtype=float32)

In [201]:
xgb.base_score


In [294]:
#50 %
xgb.predict( pd.DataFrame({'RawHigh':[.8286]}))

array([0.8291556], dtype=float32)

In [305]:
target

,RawHigh
0,0.837100
1,0.828020
2,0.830290
3,0.828780
4,0.822770
...,...
4590,0.849574
4591,0.847314
4592,0.847200
4593,0.828800
